In [ ]:
import numpy as np
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ''  # restrict GPU usage
import matplotlib.pyplot as plt
import matplotlib
import cv2
import tensorflow as tf
from tqdm.notebook import tqdm
from zstitch_gigamosaic import gmosaic
import imageio
from zstitch import get_z_step_mm

In [ ]:
# pick a sample:
sample_id = 'chair_painting'  # 'chair_painting', 'PCB', 'BGA', or 'PGA'

if sample_id == 'chair_painting':
    directory = '/data/20220213_chair_painting/'
    restore_path = '/data/20220213_green_noise_target/flat_ref_optimized_params.mat'
    tf_ckpt_path = '/data/20220213_chair_painting/CNN_ckpts'
elif sample_id == 'PCB':
    directory = '/data/20211219_PCB_160B_rechunked/'
    restore_path = '/data/20211219_green_noise_target/flat_ref_optimized_params.mat'
    tf_ckpt_path = '/data/20211219_PCB_160B_rechunked/CNN_ckpts'
elif sample_id == 'BGA':
    directory = '/data/20220207_bga_chips/'
    restore_path = '/data/20220207_green_noise_target/flat_ref_optimized_params.mat'
    tf_ckpt_path = '/data/20220207_bga_chips/CNN_ckpts'
elif sample_id == 'PGA':
    directory = '/data/20220217_pin_array'
    restore_path = '/data/20220217_green_noise_target/flat_ref_optimized_params.mat'
    tf_ckpt_path = '/data/20220217_pin_array/CNN_ckpts'
else:
    raise Exception('invalid sample_id')

use_CNN_prediction_for_3d = True  # otherwise, create sharpness map
skip_blending = False
ignore_3d = False  # if True, then only use homographic params

# these are for if use_CNN_prediction_for_3d is False; otherwise, ignored:
sigma = 8
truncate = 2

if not ignore_3d:
    z_index_ref = 'full'
else:
    z_index_ref = None
    
# CNN architecture:
filters_list = np.array([32, 32, 32, 64, 64, 64])
skip_list = [0] * len(filters_list)

## Generate gigamosaic

In [ ]:
a = gmosaic(directory, restore_path, z_index_ref=z_index_ref,
            use_CNN_prediction_for_3d=use_CNN_prediction_for_3d)
a.load_camera_parameters()
if not ignore_3d:
    a.create_network(filters_list, skip_list, tf_ckpt_path=tf_ckpt_path)
a.create_tf_dataset(prefetch=-1)

In [ ]:
# this has to be run on a GPU: scatter_nd behaves differently on GPU vs CPU in terms of out-of-bound coordinates
gigamosaic, normalize = a.generate_gigamosaic(ignore_3d=ignore_3d, sigma=sigma, truncate=truncate, margin=35,
                                              skip_blending=skip_blending)

In [ ]:
# normalization:
for i in tqdm(range(gigamosaic.shape[-1])):
    # for-loop rather than broadcasting to save memory
    gigamosaic[..., i] = gigamosaic[..., i] / (normalize + 1e-7)

In [ ]:
# unpack the tensorflow output:
gigamosaic_np = gigamosaic[:, :, :3].astype(np.uint8)
if not ignore_3d:
    height_map = gigamosaic[:, :, 3]
    if not use_CNN_prediction_for_3d:
        sharpness_map = gigamosaic[:, :, 4]

In [ ]:
# determine height range:
height_map_downsamp = height_map[::20, ::20]
cmax = height_map_downsamp[height_map_downsamp!=0].max()
cmin = height_map_downsamp[height_map_downsamp!=0].min()
crange = cmax - cmin

# height map normalized between 0 and 255 for saving as rgb image:
height_map_ = np.clip(1 - (height_map - cmin) / crange, 0, 1)  # rescale between 0 and 1
height_map_normalized = np.uint8(height_map_ * 255)

# height map in mm:
z_step_mm = get_z_step_mm(directory)
height_map_float32 = height_map_ * crange * z_step_mm

## Plot results

In [ ]:
# plot RGB gigamosaic
plt.figure(figsize=(15,15))
plt.imshow(gigamosaic_np[::20, ::20])
plt.show()

In [ ]:
# plot height map
plt.figure(figsize=(15,15))
plt.imshow(height_map_float32[::20, ::20], cmap='turbo')
plt.colorbar()
plt.show()

In [ ]:
# plot sharpness map
if not use_CNN_prediction_for_3d:
    plt.figure(figsize=(15,15))
    plt.imshow(sharpness_map[::20, ::20], cmap='turbo')
    plt.colorbar()
    plt.clim([.025, .1])
    plt.show()

## Save results

In [ ]:
# file savenames
savename_rgb = directory + '/gigamosaic_rgb'
savename_height = directory + '/gigamosaic_height'
if skip_blending:
    savename_rgb += '_no_blending'
    savename_height += '_no_blending'
if not use_CNN_prediction_for_3d:
    savename_sharpness = directory + '/gigamosaic_sharpness'
    if skip_blending:
        savename_sharpness+= '_no_blending'
    print(savename_sharpness)
print(savename_rgb)
print(savename_height)

In [ ]:
# save as tiffs (warning: big files, could be up to 24 GB each)

# save RGB stitch:
imageio.imwrite(savename_rgb + '.tif', gigamosaic_np, bigtiff=True)

# save high-precision height map:
imageio.imwrite(savename_height + '_in_mm_float32.tif', height_map_float32, bigtiff=True)

# save uint8 height map for viewing purposes:
imageio.imwrite(savename_height + '_uint8.tif', height_map_normalized, bigtiff=True)
# use up all the dynamic range of uint8 (0-255)

# save sharpness map as float32:
if not use_CNN_prediction_for_3d:
    imageio.imwrite(savename_sharpness + '.tif', sharpness_map, bigtiff=True)